<a href="https://colab.research.google.com/github/SidharthBhakth/Gensim-Tutorials/blob/master/3_Topics_and_Transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topics and Transformations

In [0]:
from pprint import pprint
from collections import defaultdict
from gensim import corpora, models

### **Creating the Corpus**

In [0]:
documents = ["Human machine interface for lab abc computer applications",
              "A survey of user opinion of computer system response time",
              "The EPS user interface management system",
              "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

In [3]:
# remove common words and tokenize
stopwords = set('for a of the and to in'.split())

texts = [ [ word for word in document.lower().split() if word not in stopwords ] for document in documents ]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1

texts = [ [token for token in text if frequency[token] > 1] for text in texts ]

# Map each word in corpus to a unique integer ID
dictionary = corpora.Dictionary(texts)

# Build corpus with BoW representation
corpus = [ dictionary.doc2bow(text) for text in texts ]

pprint(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


***Creating a transformation and transforming vectors***

In [4]:
# Initialize a tf-idf transformation
tfidf = models.TfidfModel(corpus)

# Create a simple BoW corpus and transform
doc_bow = [(0,1), (1,1)]
print(tfidf[doc_bow])

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [5]:
# Apply tf-idf transformation to a whole corpus
corpus_tfidf = tfidf[corpus]

for doc in corpus_tfidf:
  print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [0]:
# Initialize an LSI transformation
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)

# Create a double wrapper over the original corpus
# bow -> tf-idf -> fold-in LSI
corpus_lsi = lsi_model[corpus_tfidf]

In [7]:
lsi_model.print_topics(2)

[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [8]:
# Both bow->tf-idf and tf-idf->lsi transformations are actually executed on the fly
for doc, as_text in zip(corpus_lsi, documents):
  print(doc, as_text)

[(0, 0.06600783396090441), (1, -0.5200703306361848)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142738), (1, -0.7609563167700045)] A survey of user opinion of computer system response time
[(0, 0.08992639972446631), (1, -0.7241860626752505)] The EPS user interface management system
[(0, 0.0758584765217835), (1, -0.6320551586003426)] System and human system engineering testing of EPS
[(0, 0.10150299184980335), (1, -0.5737308483002954)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378303), (1, 0.16115180214026015)] The generation of random binary unordered trees
[(0, 0.8774787673119824), (1, 0.16758906864659684)] The intersection graph of paths in trees
[(0, 0.9098624686818575), (1, 0.14086553628719312)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569285), (1, -0.05392907566389181)] Graph minors A survey
